In [1]:
# Imports
import numpy as np
import itertools
from IPython.display import display, Markdown, Latex, Math

%matplotlib notebook

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from matplotlib.patches import Circle, Wedge
from matplotlib.collections import PatchCollection

import tropical
import data_gen2

import itertools

import time

from scipy.spatial import ConvexHull, Delaunay
from scipy.spatial.qhull import QhullError

from scipy.optimize import linprog

In [2]:
def evaluate_trop(polynomial, data):

    temp = np.ones((data.shape[0], 1))
    test = np.concatenate((data, temp), axis=1)
    
    Fresult = np.matmul(polynomial, test.T)
    Fmax = np.amax(Fresult, axis=0)
    Flocs = np.argmax(Fresult, axis=0)
    
    return Fmax, Flocs

In [18]:
# Get initial F and G populations and the problem

nTerms = 10000

Fterms = np.random.uniform(low=-10, high=10, size=(nTerms, 3))
Gterms = np.random.uniform(low=-10, high=10, size=(nTerms, 3))

termsToUse = 20
popSize = 20

epochs = 5

structure = 'xor'

if structure == 'circles':
    num_samples = 4
    
    r1 = 0.5 # Radius of inner circle
    data_func = lambda n: data_gen2.circles(n, r1)
elif structure == 'xor':
    num_samples = 2
    
    r1 = 0.25
    data_func = lambda n: data_gen2.harderXor(n, r1=r1, alpha=np.sqrt(2)-1)

In [19]:
# Set up validation and plotting sets
thing = 101
x = np.linspace(-1, 1, thing)
y = np.linspace(-1, 1, thing)

X, Y = np.meshgrid(x, y)

x = np.reshape(X, thing**2)
y = np.reshape(Y, thing**2)

r1sqr = r1**2
r2sqr = 2*r1sqr
r3sqr = 3*r1sqr

X_test = np.vstack((x, y)).T

# Circle
if structure == 'circles':
    indices = X_test[:, 0]**2 + X_test[:, 1]**2 < r1sqr
    X_trainCirc = X_test[indices, :]
    y_trainCirc = np.ones((X_trainCirc.shape[0], 1))

    indices = np.logical_and(r2sqr < X_test[:, 0]**2 + X_test[:, 1]**2, X_test[:, 0]**2 + X_test[:, 1]**2 < r3sqr)
    X_trainCirc2 = X_test[indices, :]
    y_trainCirc2 = -np.ones((X_trainCirc2.shape[0], 1))

    X_testCirc = np.vstack((X_trainCirc, X_trainCirc2))
    y_testCirc = np.vstack((y_trainCirc, y_trainCirc2))
elif structure == 'xor':
    X_testCircs = []
    y_testCircs = []

    for loc in [[0.5, 0.5], [-0.5, 0.5], [0.5, -0.5], [-0.5, -0.5]]:
        indices = (X_test[:, 0] - loc[0])**2 + (X_test[:, 1] - loc[1])**2 < r1sqr
        X_testCircs.append(X_test[indices, :])
        if loc[0] == loc[1]:
            y_testCircs.append(np.ones((X_testCircs[-1].shape[0], 1)))
        else:
            y_testCircs.append(-np.ones((X_testCircs[-1].shape[0], 1)))

        indices = np.logical_and(r2sqr < (X_test[:, 0] - loc[0])**2 + (X_test[:, 1] - loc[1])**2, (X_test[:, 0] - loc[0])**2 + (X_test[:, 1] - loc[1])**2 < r3sqr)

        X_testCircs.append(X_test[indices, :])
        if loc[0] == loc[1]:
            y_testCircs.append(-np.ones((X_testCircs[-1].shape[0], 1)))
        else:
            y_testCircs.append(np.ones((X_testCircs[-1].shape[0], 1)))

    X_testCirc = np.vstack(X_testCircs)
    y_testCirc = np.vstack(y_testCircs)

In [20]:
# Create evaluation function

def eval_func(fGenome, gGenome):
    global Fterms
    global Gterms
    global X_testCirc
    global y_testCirc
    F, _ = evaluate_trop(Fterms[fGenome, :], X_testCirc)
    G, _ = evaluate_trop(Gterms[gGenome, :], X_testCirc)
    result = F - G
    
    return np.sum((result.reshape((-1, 1)) > 0) == (y_testCirc > 0))/(len(y_testCirc)) # Percent correct
    #np.mean(np.power(y_testCirc - result.reshape((-1, 1)), 2)) # MSE

In [24]:
initialPopulationF = []
initialPopulationG = []
for i in range(popSize):
    fGenome = np.random.choice(range(nTerms), termsToUse//2)
    gGenome = np.random.choice(range(nTerms), termsToUse//2)
    
    initialPopulationF.append((fGenome, None))
    initialPopulationG.append((gGenome, None))
print(eval_func(fGenome, gGenome))

0.528585757272
